# Measuring the impact of student research labs at the University of Texas 

In [57]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  black solid !important;
  color: black !important;
}
</style>

In [70]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
import janitor 
import statsmodels.formula.api as smf
import censusdata 
%matplotlib inline
pd.set_option('display.max_colwidth', 100)

In [82]:
data = pd.read_excel("IDA Student Info Fall 2018 - Test.xlsx", sheet_name= "Sheet1").clean_names()

pd.set_option("display.max_columns", 101)
pd.set_option("display.max_rows", 10)
pd.set_option('display.min_rows', 10)
pd.set_option('display.max_colwidth', 25)


# Codebook 
### \*Currently missing columns with individual identifiers\*
| **Variable Name** | **Plain English**|
| --- | --- |
|sch_maj1| School Major |
|ut_abbr1| UT abbreviation |
|cola_abbr1| COLA abbreviation|
|profiles| There's a lot of information in this column that I don't think is important|
|maj1_name| Name of major |
|sch_maj2| Second major|
|ut_abbr2| UT Abbreviation of second major |
|cola_abbr2| COLA abbreviation of second major|
|maj2_name|Second Major Name|
|cumulative_gpa |Cumulative GPA|
|gpa_hrs | GPA hours completed at UT (includes currently taking)|
|credit_hrs_excludes_current_ |Excludes currently taking, but it includes transfer credits|
|current_hrs|Current hours taking this semester|
|perm_street_2|Second Address|
|perm_street_3|Third Address|
|perm_city| Permanent City|
|perm_state| Permanent State|
|perm_postal_code| Postal Code|
|citizenship| Citizenship |
|gender|gender|
|ethnicities| Ethnicities|
|classification| Freshman/Sophmore etc.|
|ut_degrees| UT degrees if they've already been awarded one. (I think)|
|current_ageas_of_today| Current Age as of last draw of the data|
|cumulative_gpa_in_residenceunavailable_if_current_ccyys| Cumulative GPA in residence (unavailable if taking classes this semester)|
|cumulative_hrs_incomplete|Cumulative hours completed|
|cumulative_hrs_transferred|Cumulative hours transfered|
|sem_gpa_fall_2018unavailable_if_current_ccyys|Semester GPA Fall 2018 (unavailable if current semester)|
|sem_gpa_hrs_fall_2018unavailable_if_current_ccyys|Semester GPA and Hours Fall 2018 (unavailable if current semester)|
|overall_act_score| ACT Score|
|sat_verbal_score| SAT Verbal Score |
|sat_quant_score| SAT Quant Score |
|semester_hrs_incomplete_fall_2018_unavailable_if_current_ccyys| Semester hours incomplete (unavailable if current semester)|
|fatherseducation|Father's education|
|motherseducation|Mother's education|
|familyincome|Family income category|
|high_school|High School|
|hs_rank | High School Rank|
|unofficial_hnrs_pgm_s_ |Unofficial Honors Program|
|popestimate2018 | City Population Estimate from the Census |
|c_rating | High School Accountability Rating (TEA) |


# Data Cleaning 

#### UT data

There are 10599 students in the data for 2018. Removing non-texas residents there are 9743. 

In [89]:
df = data.drop(['sch_maj2', 'ut_abbr2','cola_abbr2','maj2_name','cumulative_gpa_in_residenceunavailable_if_current_ccyys',
             'sem_gpa_fall_2018unavailable_if_current_ccyys','sem_gpa_hrs_fall_2018unavailable_if_current_ccyys',
             'unofficial_hnrs_pgm_s_','semester_hrs_incomplete_fall_2018_unavailable_if_current_ccyys','cumulative_hrs_incomplete', 'ut_degrees', 'cola_abbr1',
              'perm_street_3','perm_street_2','profiles'], axis=1)

df = df.rename(columns={"credit_hrs_excludes_current_": "credit_hrs_ex_cur", "current_ageas_of_today": "age", "cumulative_hrs_transferred" : "hrs_transferred"})



df['hs_rank'] = df['hs_rank'].astype(str)
df['overall_act_score'] = df['overall_act_score'].astype(str)
df['age'] = df['age'].astype(str)
df['sat_verbal_score'] = df['sat_verbal_score'].astype(str)
df['sat_quant_score'] = df['sat_quant_score'].astype(str)

df['age'] = df['age'].str.replace("\(R\)", "")
df['age'] = df['age'].str.replace("\*Minor", "")

df['classification'] = df['classification'].replace("\(R\)", "")
df = df.replace('unknown', "0/0")
df = df.replace('None', "0")

df['sat_verbal_score'] = df['sat_verbal_score'].astype(int)
df['sat_quant_score'] = df['sat_quant_score'].astype(int)
df['overall_act_score'] = df['overall_act_score'].astype(int)
df['age'] = df['age'].astype(int)

df[['numerator','denominator']] = df.hs_rank.str.split("/",expand=True,)
df['numerator'] = df['numerator'].astype(int)
df['denominator'] = df['denominator'].astype(int)
df['hs_rank'] = df['numerator']/df['denominator']
df['hs_rank'] = df['hs_rank']*100
df = df.drop(['numerator','denominator'], axis = 1)

df = df[df['perm_state'] == 'TX']
df['high_school'] = df['high_school'].str.replace('\d+', '')
df['high_school'] = df['high_school'].str.replace(r" \(.*\)", '')
df['high_school'] = df['high_school'].str.replace('hs', 'High School')
df['high_school'] = df['high_school'].str.replace('h s', 'High School')
df['high_school'] = df['high_school'].str.replace('HS', 'High School')
df['high_school'] = df['high_school'].str.replace('H S', 'High School')
df['high_school'] = df['high_school'].str.replace('Hs', 'High School')
df


,sch_maj1,ut_abbr1,maj1_name,cumulative_gpa,gpa_hrs,credit_hrs_ex_cur,current_hrs,perm_city,perm_state,perm_postal_code,citizenship,gender,ethnicities,classification,age,hrs_transferred,overall_act_score,sat_verbal_score,sat_quant_score,fatherseducation,motherseducation,familyincome,high_school,hs_rank
0,L 23400,C C,Classical Studies,3.1717,68,77,18,Lampasas,TX,76550-3612,U. S. Citizen,Male,Black or African Amer...,Junior,20,0,24,550,600,High School Diploma O...,Some College,0/0,Lampasas High School,6.048387
1,L 50600 (R),HIS,History,2.0767,43,76,17 (R),Austin,TX,78728-4418,U. S. Citizen,Female,Asian,Junior (R),21,35,23,560,590,Graduate Or Professio...,Graduate Or Professio...,H,Westwood High School,69.798658
2,L 00300,NaN,College of Liberal Ar...,4.0000,3,3,3,Austin,TX,78704-7957,U. S. Citizen,Female,Hispanic,Senior,32,0,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,C,Homer Hanna High School,NaN
3,L 55000,IRG,International Relatio...,3.5208,34,79,15,Spring,TX,77388-5765,U. S. Citizen,Female,"Hispanic, Asian",Junior,20,42,22,660,540,Graduate Or Professio...,Associate's Degree,G,Klein Collins High Sc...,45.080946
4,L 47500,GOV,Government,0.0000,0,0,13,Lewisville,TX,75057-5123,U. S. Citizen,Female,Black or African Amer...,Freshman,18,0,0,700,600,Bachelor's Or Four-ye...,Educational Level Unk...,A,Lewisville High School,7.200720
5,L 83500,PSY,Psychology (BA),2.8974,55,64,15,Houston,TX,77082-2470,U. S. Citizen,Female,Asian,Junior,20,0,22,630,500,Some High School,Graduate Or Professio...,B,Pasadena Memorial Hig...,4.264099
6,L 96000,WGS,Women's and Gender St...,3.6368,22,23,14,Carrollton,TX,75010-3227,U. S. Citizen,Female,Asian,Freshman,19,0,29,0,0,Graduate Or Professio...,Associate's Degree,0/0,Hebron High School,23.240223
7,L 31500,ECO,Economics,3.7487,24,56,13,Austin,TX,78705-4432-01,Non - U. S. Citizen,Female,NaN,Sophomore,20,27,0,530,640,0/0,0/0,0/0,Lyceum School,NaN
8,L 31500,ECO,Economics,3.4375,48,101,14,Austin,TX,78733-2003,U. S. Citizen,Female,"Asian, White",Senior,21,53,28,630,660,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,F,Anderson,NaN
9,L 00100,NaN,Undeclared,0.0000,0,50,12,Houston,TX,77095-3700,U. S. Citizen,Female,"Hispanic, White",Sophomore,19,50,28,630,630,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,F,Cypress Falls High Sc...,7.479862


### The Ethnicities column provides an exhaustive list of an individual's ethnicities. Below I changed the values of individuals with multiple ethnicities to "mixed race". 

In [84]:
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Black or African American','ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Black or African American, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, Asian', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Black or African American, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, Black or African American, Native Hawaiian or Other Pacific Islander, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, American Indian or Alaska Native, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'American Indian or Alaska Native, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian, White', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Native Hawaiian or Other Pacific Islander', 'ethnicities'] = "Mixed Race"
df.loc[df['ethnicities'] == 'Hispanic, Asian', 'ethnicities'] = "Mixed Race"

### Census Data on city/town population 

In [90]:
pop = pd.read_csv('sub-est2018_all.csv', encoding='latin-1').clean_names()
pd.set_option("display.max_rows", 15)
pd.set_option('display.min_rows', 15)
pop.head()

texas = pop[pop['stname'] == 'Texas']
texas['name'] = texas['name'].str.replace(" city", "")
texas['name'] = texas['name'].str.replace(" town", "")





C:\Users\tenis\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\tenis\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Texas Accountability Ratings 

[Click here for data dictionary](https://rptsvr1.tea.texas.gov/perfreport/account/2019/download/camprate.html)

In [86]:
ratings = pd.read_csv("CREF.csv").clean_names()
cols = ['grdspan']
for col in cols:
    ratings[col] = ratings[col].map(lambda x: str(x).lstrip('\'').rstrip('\''))

hs_ratings = ratings[(ratings['grdspan'] == '09 - 12') | (ratings['grdspan'] == '08 - 12') | (ratings['grdspan'] == '10 - 12') |
                    (ratings['grdspan'] == '11 - 12') | (ratings['grdspan'] == '12 - 12') | (ratings['grdspan'] == '07 - 12') |
                    (ratings['grdspan'] == '06 - 12') | (ratings['grdspan'] == '05 - 12') | (ratings['grdspan'] == '04 - 12') |
                    (ratings['grdspan'] == '03 - 12') | (ratings['grdspan'] == '02 - 12') | (ratings['grdspan'] == '01 - 12') |
                    (ratings['grdspan'] == 'kg - 12') | (ratings['grdspan'] == 'PK - 12') | (ratings['grdspan'] == 'EE - 12')]


hs_ratings['campname'] = hs_ratings['campname'].str.replace("H S", "High School")
hs_ratings['campname'] = hs_ratings['campname'].str.title()
hs_ratings = hs_ratings[['campname', 'c_rating']]
hs_ratings


C:\Users\tenis\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
C:\Users\tenis\anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,campname,c_rating
0,Cayuga High School,A
3,Elkhart High School,B
8,Frankston High School,B
11,Neches High School,A
13,Palestine High School,B
19,Westwood High School,B
23,Slocum High School,A
...,...,...
8816,Graham High School,B
8817,Graham Learning Ctr,Not Rated


### Joining Dataframes

In [88]:
df.loc[(df.high_school == 'Anderson'),'high_school']= 'Anderson High School'

joined = pd.merge(df,
                 texas[['name', 'popestimate2018']],
                 left_on= 'perm_city',
                 right_on= 'name',
                 how = 'left')
joined = pd.merge(joined, 
                 hs_ratings, 
                 left_on = 'high_school',
                 right_on = 'campname',
                 how = 'left')


pd.set_option("display.max_rows", 100)
pd.set_option('display.min_rows', 100)
null_data = joined[joined['c_rating'].isnull()]
null_data

,sch_maj1,ut_abbr1,maj1_name,cumulative_gpa,gpa_hrs,credit_hrs_ex_cur,current_hrs,perm_city,perm_state,perm_postal_code,citizenship,gender,ethnicities,classification,age,hrs_transferred,overall_act_score,sat_verbal_score,sat_quant_score,fatherseducation,motherseducation,familyincome,high_school,hs_rank,name,popestimate2018,campname,c_rating
4,L 00300,NaN,College of Liberal Ar...,4.0000,3,3,3,Austin,TX,78704-7957,U. S. Citizen,Female,Hispanic,Senior,32,0,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,C,Homer Hanna High School,NaN,Austin,964254.0,NaN,NaN
9,L 31500,ECO,Economics,3.7487,24,56,13,Austin,TX,78705-4432-01,Non - U. S. Citizen,Female,NaN,Sophomore,20,27,0,530,640,0/0,0/0,0/0,Lyceum School,NaN,Austin,964254.0,NaN,NaN
17,L 39300,E,English,3.0009,63,122,15,Bulverde,TX,78163-4112,U. S. Citizen,Female,Mixed Race,Senior,22,59,0,0,0,Some College,Bachelor's Or Four-ye...,F,SmitHigh Schoolon Val...,13.480885,Bulverde,5240.0,NaN,NaN
18,L 31500,ECO,Economics,3.6022,83,107,15,Katy,TX,77450-5553,U. S. Citizen,Female,Black or African Amer...,Senior,22,21,0,720,680,Graduate Or Professio...,Graduate Or Professio...,0/0,Watford Grammar School,NaN,Katy,19216.0,NaN,NaN
19,L 83500,PSY,Psychology (BA),3.0180,55,94,12,Whitehouse,TX,75791-5808,U. S. Citizen,Female,White,Senior,21,39,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,E,East Texas Christian ...,NaN,Whitehouse,8799.0,NaN,NaN
20,L 83500,PSY,Psychology (BA),3.0180,55,94,12,Whitehouse,TX,75791-5808,U. S. Citizen,Female,White,Senior,21,39,0,0,0,Bachelor's Or Four-ye...,Bachelor's Or Four-ye...,E,East Texas Christian ...,NaN,Whitehouse,8799.0,NaN,NaN
24,L 76400,PHL,Philosophy,0.0000,0,57,15,Odessa,TX,79762-7644,U. S. Citizen,Female,Mixed Race,Sophomore,20,57,0,600,490,High School Diploma O...,Bachelor's Or Four-ye...,B,Odessa Senior High Sc...,16.536458,Odessa,120568.0,NaN,NaN
28,L 55000 (R),IRG,International Relatio...,3.0894,97,98,12 (R),Austin,TX,78741-1804-01,Non - U. S. Citizen,Female,NaN,Senior (R),21,0,0,510,430,0/0,0/0,0/0,Ss John Neumann/Maria...,32.666667,Austin,964254.0,NaN,NaN
31,L 58000,J S,Jewish Studies,2.8235,51,78,15,Dallas,TX,75252-5478,U. S. Citizen,Female,Mixed Race,Junior,22,30,22,0,0,Graduate Or Professio...,Graduate Or Professio...,F,Emery Weiner School,NaN,Dallas,1345047.0,NaN,NaN
32,L 48500,H S,Health and Society,3.7455,34,74,15,Austin,TX,78746-2524,U. S. Citizen,Female,White,Junior,20,34,29,720,610,Graduate Or Professio...,Bachelor's Or Four-ye...,H,St Andrews Episcopal ...,NaN,Austin,964254.0,NaN,NaN


# Statistical Analysis 

In [12]:
model = smf.ols("cumulative_gpa ~ citizenship + age + sat_verbal_score + sat_quant_score + hs_rank + motherseducation + fatherseducation + ethnicities", df)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         cumulative_gpa   R-squared:                       0.120
Model:                            OLS   Adj. R-squared:                  0.117
Method:                 Least Squares   F-statistic:                     37.99
Date:                Tue, 24 Mar 2020   Prob (F-statistic):          1.69e-185
Time:                        11:27:06   Log-Likelihood:                -12792.
No. Observations:                7522   AIC:                         2.564e+04
Df Residuals:                    7494   BIC:                         2.583e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
============================================================================================================================
                                                               coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------------------
Intercept                                                   -0.4088      0.359     -1.138      0.255      -1.113       0.296
citizenship[T.Permanent Resident Alien]                     -0.2062      0.187     -1.103      0.270      -0.573       0.160
citizenship[T.U. S. Citizen]                                -0.2026      0.182     -1.114      0.265      -0.559       0.154
motherseducation[T.Associate's Degree]                      -0.5972      0.782     -0.763      0.445      -2.131       0.936
motherseducation[T.Bachelor's Or Four-year Degree]          -0.5704      0.781     -0.731      0.465      -2.101       0.960
motherseducation[T.Educational Level Unknown]               -1.3347      0.788     -1.694      0.090      -2.879       0.210
motherseducation[T.Graduate Or Professional Degree]         -0.5525      0.781     -0.708      0.479      -2.083       0.978
motherseducation[T.High School Diploma Or Equivalent]       -0.7448      0.782     -0.953      0.341      -2.277       0.787
motherseducation[T.No High School]                          -0.9487      0.785     -1.208      0.227      -2.488       0.590
motherseducation[T.Some College]                            -0.6979      0.781     -0.894      0.371      -2.228       0.832
motherseducation[T.Some High School]                        -1.0395      0.782     -1.330      0.184      -2.572       0.493
fatherseducation[T.Associate's Degree]                       1.4660      0.777      1.886      0.059      -0.057       2.989
fatherseducation[T.Bachelor's Or Four-year Degree]           1.5201      0.775      1.962      0.050       0.001       3.039
fatherseducation[T.Educational Level Unknown]                2.0836      0.783      2.661      0.008       0.549       3.618
fatherseducation[T.Graduate Or Professional Degree]          1.6324      0.774      2.108      0.035       0.114       3.151
fatherseducation[T.High School Diploma Or Equivalent]        1.4942      0.775      1.927      0.054      -0.025       3.014
fatherseducation[T.No High School]                           1.6313      0.779      2.095      0.036       0.105       3.158
fatherseducation[T.Some College]                             1.6239      0.775      2.095      0.036       0.104       3.143
fatherseducation[T.Some High School]                         1.6814      0.777      2.164      0.030       0.158       3.205
ethnicities[T.Asian]                                        -0.7943      0.471     -1.685      0.092      -1.718       0.130
ethnicities[T.Black or African American]                    -0.9174      0.475     -1.933      0.053      -1.848       0.013
ethnicities[T.Hispanic]                                     -0.9222      0.474     

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10599 entries, 0 to 10598
Data columns (total 24 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sch_maj1           10599 non-null  object 
 1   ut_abbr1           10088 non-null  object 
 2   maj1_name          10599 non-null  object 
 3   cumulative_gpa     10599 non-null  float64
 4   gpa_hrs            10599 non-null  int64  
 5   credit_hrs_ex_cur  10599 non-null  int64  
 6   current_hrs        10599 non-null  object 
 7   perm_city          10584 non-null  object 
 8   perm_state         10192 non-null  object 
 9   perm_postal_code   10529 non-null  object 
 10  citizenship        10599 non-null  object 
 11  gender             10599 non-null  object 
 12  ethnicities        9834 non-null   object 
 13  classification     10599 non-null  object 
 14  age                10599 non-null  int32  
 15  hrs_transferred    10599 non-null  int64  
 16  overall_act_score  105